In the first cell, we define our query function. This is to be copied and pasted each time we want to query science windows. 

In [29]:
import time

import astroquery.heasarc
from astropy.coordinates import SkyCoord
from astropy import units as u

Heasarc = astroquery.heasarc.Heasarc()

def get_scw_list(ra_obj, dec_obj,radius,start_date,end_date ):
    R = Heasarc.query_region(
            position = SkyCoord(ra_obj, dec_obj, unit='deg'),
            radius = f"{radius} deg",
            catalog = 'intscw', # 'mission' has been deprecated
            time = start_date + " .. " + end_date,
            good_isgri = ">1000",
        )
    # print(R)
    R.sort('scw_id') # changed SCW_ID to scw_id

    return R['scw_id'], R['scw_ver']  # changed SCW_VER to scw_ver


assert astroquery.__version__ >= '0.4.2.dev6611'
assert 'isdc' in astroquery.heasarc.Conf.server.cfgtype

In this second cell, define our source and region to query.

In [30]:
from astroquery.simbad import Simbad
from astropy import coordinates as coord

import json

import os
import shutil
import random

from astropy.io import fits

from scipy import stats

source_name = 'Crab'

result_table = Simbad.query_object(source_name)
source_coord = coord.SkyCoord(result_table['ra'][0], result_table['dec'][0], unit=("hourangle", "deg")) # changed RA and DEC to ra and dec respectively
ra=source_coord.ra.deg
dec=source_coord.dec.deg
radius=8.

Finally, define the time interval within which we wish to retrieve our object.

In [31]:
scwlist,version = get_scw_list(ra,dec, 10, '2020-01-01T00:00:00', '2020-06-01T00:00:00')

We then pick 10 science windows at random. 

In [32]:
nscw = 10
if len(scwlist) == 0:
    raise Exception('No science windows')

scw_pick = [(s+"."+v).strip() for s,v in zip(scwlist,version)]

if nscw >0 and len(scw_pick)>=nscw:
    random.seed(0)
    scw_pick = random.sample(scw_pick, nscw)
scw_pick = sorted(scw_pick)

print(scw_pick)

['021200630021.001', '074900350021.001', '110900000021.001', '154400530021.001', '155000270020.001', '155100280090.001', '155300030072.001', '176400540010.001', '182100530031.001', '231400550021.001']


Before moving on to using this query for our ODA, we need to input our token. 

In [33]:
token = ''

import getpass
token = getpass.getpass('Insert the token')

In [34]:
import oda_api.token
token = oda_api.token.discover_token()

In [35]:
oda_api.token.decode_oda_token(token)

{'sub': 'hucklethums@icloud.com',
 'email': 'hucklethums@icloud.com',
 'name': 'interstellxr',
 'roles': 'authenticated user, public-pool-hpc',
 'exp': 1741191208}

Now let us look at an example.

In [38]:
from oda_api.api import DispatcherAPI

disp = DispatcherAPI(url="https://www.astro.unige.ch/mmoda/dispatch-data")

par_dict = {
"E1_keV": "28",
"E2_keV": "40",
"detection_threshold": "7",
"instrument": "isgri",
"osa_version": "OSA11.2",
"product": "isgri_image",
"product_type": "Real",
"scw_list": scw_pick,
'token': token
}
# "integral_data_rights": "all-private" had to be removed as didn't have the rights

data_collection = disp.get_product(**par_dict)
data_collection.show()
data_collection.mosaic_image_0_mosaic.show()
print(data_collection.mosaic_image_0_mosaic.show_meta())


please beware that by default, in a typical setup, oda_api will not output much. To learn how to increase the verbosity, please refer to the documentation: https://oda-api.readthedocs.io/en/latest/user_guide/ScienceWindowList.html?highlight=logging#Let's-get-some-logging . 
To disable this message you can pass `.get_product(..., silent=True)`


ID=0 prod_name=mosaic_image_0_mosaic  meta_data: {'product': 'mosaic', 'instrument': 'isgri', 'src_name': '', 'query_parameters': None}

ID=1 prod_name=dispatcher_catalog_1  meta_data: 

------------------------------
name: mosaic_image
meta_data dict_keys(['product', 'instrument', 'src_name', 'query_parameters'])
number of data units 6
------------------------------
data uniti 0 ,name: PRIMARY
data uniti 1 ,name: GROUPING
data uniti 2 ,name: ISGR-MOSA-IMA
data uniti 3 ,name: ISGR-MOSA-IMA
data uniti 4 ,name: ISGR-MOSA-IMA
data uniti 5 ,name: ISGR-MOSA-IMA
------------------------------
product : mosaic
instrument : isgri
src_name : 
query_parameters : None
------------------------------
None


In [44]:
import matplotlib.pyplot as plt

from oda_api.plot_tools import OdaImage

im=OdaImage(data_collection.mosaic_image_0_mosaic)
im.show(unit_ID=1)


No instrument in data collection


Exception: No mosaic extension in data product